In [183]:
import pandas as pd
from datetime import timedelta
import matplotlib.pyplot as plt
import pytz
import calendar

In [ ]:
df = pd.read_csv('forest-of-sivakar.csv')
print(df.tail())


# Preprocessing

### Split sessions that cross midnight

In [ ]:
# Drop unnecessary columns
columns_to_drop = ['Note', 'Tree Type', 'Is Success']
df = df.drop(columns=columns_to_drop)


def parse_timestamp(ts):
    dt = pd.to_datetime(ts, format='%a %b %d %H:%M:%S GMT%z %Y')
    return dt.tz_convert(pytz.timezone('Asia/Kolkata'))

def split_at_midnight(row):
    start = row['Start Time']
    end = row['End Time']

    local_tz = start.tz
    midnight = (start + timedelta(days=1)).replace(hour=0, minute=0, second=0, microsecond=0)

    if start.date() != end.date():
        first_half = pd.Series({
            'Start Time': start,
            'End Time': (midnight - timedelta(seconds=1)).tz_convert(local_tz)
        })

        second_half = pd.Series({
            'Start Time': midnight.tz_convert(local_tz),
            'End Time': end
        })

        return [first_half, second_half]

    return [row]

# Parse timestamps
df['Start Time'] = df['Start Time'].apply(parse_timestamp)
df['End Time'] = df['End Time'].apply(parse_timestamp)

# Apply the split_at_midnight function and explode the result
split_rows = []
for _, row in df.iterrows():
    split_rows.extend(split_at_midnight(row))
df = pd.DataFrame(split_rows)
df.reset_index()

# Display the first few rows of the processed dataframe
print(df.head(10))

# Optionally, save the processed data to a new CSV file
# df.to_csv('processed_data.csv', index=False)

In [ ]:
df.info()

### Calculate Duration

In [187]:
# Calculate duration for each entry
df['Duration'] = df['End Time'] - df['Start Time']

### Set derived time fields

In [ ]:
df['Year-Month'] = df['Start Time'].dt.to_period('M')
df['Date'] = df['Start Time'].dt.date
df['Day of Week'] = df['Start Time'].dt.dayofweek

## Tag Categories

In [189]:
reading_tags = [
  'Book',
  'Reading Fiction',
  'Reading non-fiction',
]

career_tags = [
  'Virtusa',
  'Freelancing (Revocare)',
  'Freelancing (Totalley)',
  'Career',
  'My projects',
  'Computer Science',
  'TreeDots'
]

neuroplasticity_tags = [
  'French',
  'German',
  'Hindi',
  'Brain Training',
  'Music'
]

In [190]:
tag_to_category = {}

for tag in reading_tags:
    tag_to_category[tag] = 'Reading'

for tag in career_tags:
    tag_to_category[tag] = 'Career'

for tag in neuroplasticity_tags:
    tag_to_category[tag] = 'Neuroplasticity'

df['Category'] = df['Tag'].map(tag_to_category)

# Analytics

In [191]:
def get_monthly_hours(df, tags=None):
  if tags:
    subset = df[df['Tag'].isin(tags)]
  else:
    subset = df

  # Group by year-month and sum the durations, then convert to hours
  subset_monthly_totals = subset.groupby('Year-Month')['Duration'].sum().dt.total_seconds() / 3600

  # Round the hours to 2 decimal places
  subset_monthly_totals = subset_monthly_totals.round(2)

  # The result is now a Series with Year-Month as index and Total Hours as values
  return subset_monthly_totals

In [192]:
def plot_monthly_hours(monthly_hours_df):
  # Set up the plot
  plt.figure(figsize=(20, 6))  # Adjust the width (20) and height (6) as needed

  # Create a color map for years
  years = monthly_hours_df.index.year.unique()
  color_map = plt.colormaps['tab10']  # You can change 'tab10' to other color maps
  colors = {year: color_map(i/len(years)) for i, year in enumerate(years)}

  # Create lists to hold x-positions and colors for each bar
  x_positions = []
  bar_colors = []
  labels = []

  # Prepare data for plotting
  for i, (date, value) in enumerate(monthly_hours_df.items()):
      month = date.month
      year = date.year
      x_positions.append(month + (year - min(years)) * 12)
      bar_colors.append(colors[year])
      labels.append(f"{calendar.month_abbr[month]}\n{year}")

  # Plot bars
  plt.bar(x_positions, monthly_hours_df.values, color=bar_colors, width=0.8)

  # Customize the plot
  plt.title('Hours per Month', fontsize=16)
  plt.xlabel('Month-Year', fontsize=12)
  plt.ylabel('Total Hours', fontsize=12)
  plt.xticks(x_positions, labels, rotation=45, ha='right')

  # Add legend
  for year in years:
      plt.bar(0, 0, color=colors[year], label=str(year))
  plt.legend(title='Year', bbox_to_anchor=(1.05, 1), loc='upper left')

  plt.tight_layout()

  # Show the plot
  plt.show()

## Total Time Each Year

In [ ]:

# Extract year from Start Time
df['Year'] = df['Start Time'].dt.year

# Group by year and sum the durations, then convert to hours
yearly_totals = df.groupby('Year')['Duration'].sum().dt.total_seconds() / 3600

# Round the hours to 2 decimal places
yearly_totals = yearly_totals.round(2)

# The result is now a Series with Year as index and Total Hours as values
print(yearly_totals)

In [ ]:
yearly_totals.plot.bar()

## Total Time Each Month

In [ ]:
monthly_totals = get_monthly_hours(df)
monthly_totals.head()

In [ ]:
plot_monthly_hours(monthly_totals)

## Distribution of Daily Totals

In [ ]:
# Group by date and sum the durations, then convert to minutes
daily_totals = df.groupby('Date')['Duration'].sum().dt.total_seconds() / 3600

# Round the minutes to 2 decimal places
daily_totals = daily_totals.round(2)

# The result is now a Series with Date as index and Total Minutes as values
print(daily_totals.tail())

In [ ]:
plt.figure(figsize=(12, 6))

plt.hist(daily_totals, bins=range(0, int(daily_totals.max()) + 2, 1),
         edgecolor='black', align='left')

plt.title('Distribution of Daily Total Hours', fontsize=16)
plt.xlabel('Hours', fontsize=12)
plt.ylabel('Frequency', fontsize=12)

plt.xticks(range(0, int(daily_totals.max()) + 2, 1))

plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

# Analytics by Tags

In [ ]:
tags_counts = df['Tag'].value_counts()
tags_counts.head(20)

In [ ]:
tags_counts.plot.bar()

## Journaling

In [201]:
journaling_monthly_totals = get_monthly_hours(df, ['Journaling'])

In [ ]:
journaling_monthly_totals.head()

In [ ]:
plot_monthly_hours(journaling_monthly_totals)

## Reading

In [204]:
reading_monthly_totals = get_monthly_hours(df, reading_tags)

In [ ]:
reading_monthly_totals.head()

In [ ]:
plot_monthly_hours(reading_monthly_totals)

## Career

In [207]:
career_monthly_totals = get_monthly_hours(df, career_tags)

In [ ]:
career_monthly_totals.head()

In [ ]:
plot_monthly_hours(career_monthly_totals)

## Neuroplasticity

In [210]:
neuroplasticity_monthly_totals = get_monthly_hours(df, neuroplasticity_tags)

In [ ]:
neuroplasticity_monthly_totals.tail()

In [ ]:
plot_monthly_hours(neuroplasticity_monthly_totals)

## Top Tag For Each Day

In [ ]:
# Group by Date and Tag, sum the duration
grouped = df.groupby(['Date', 'Tag'])['Duration'].sum().reset_index()

# Find the tag with maximum duration for each date
top_tags = grouped.loc[grouped.groupby('Date')['Duration'].idxmax()]

# Sort by date
top_tags = top_tags.sort_values('Date')

# Reset index for clean output
top_tags = top_tags.reset_index(drop=True)

# Display the result
print(top_tags)

In [ ]:
top_tags['Tag'].value_counts().head()

In [ ]:
top_tags['Tag'].value_counts().plot.bar()

## Top Tags for Month

In [ ]:
# Group by Year-Month and Tag, sum the duration
grouped = df.groupby(['Year-Month', 'Tag'])['Duration'].sum().reset_index()

# Find the tag with maximum duration for each month
top_tags_monthly = grouped.loc[grouped.groupby('Year-Month')['Duration'].idxmax()]

# Sort by Year-Month
top_tags_monthly = top_tags_monthly.sort_values('Year-Month')

# Reset index for clean output
top_tags_monthly = top_tags_monthly.reset_index(drop=True)

# Convert Year-Month to string for better readability
top_tags_monthly['Year-Month'] = top_tags_monthly['Year-Month'].astype(str)

# Display the result
print(top_tags_monthly)

# If you want to save this to a CSV file, uncomment the following line:
# top_tags_monthly.to_csv('top_tags_monthly.csv', index=False)

In [ ]:
top_tags_monthly['Tag'].value_counts()

In [ ]:
top_tags_monthly['Tag'].value_counts().plot.bar()